In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import plotly
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
def visualize_normal_mode(geometry, eigenvector, scale=10):
    vec = eigenvector.reshape(-1, 3)
    trace2 = go.Cone(
        x=geometry[:, 0],
        y=geometry[:, 1],
        z=geometry[:, 2],
        u=vec[:, 0] * scale,
        v=vec[:, 1] * scale,
        w=vec[:, 2] * scale,
        sizemode="absolute", # "absolute"
        sizeref=2,
        anchor="tail"
    )
    return [trace2]

def visualize_normal_modes(geometry, eigenvectors, scale=10, cols=3):
#     eigenvectors = eigenvectors.round(2)
    N3, N3 = eigenvectors.shape
    rows = int(N3/cols)
    if N3 % cols > 0:
        rows += 1
    specs = [[{'is_3d': True} for i in range(cols)]
             for j in range(rows)]
    fig = make_subplots(rows=rows, cols=cols, specs=specs)
    for row in range(rows):
        for col in range(cols):
            i = row * cols + col
            if i >= N3:
                continue
            traces = visualize_normal_mode(geometry, eigenvectors[:, i], scale)
            fig.add_trace(traces[0], row=row + 1, col=col + 1)
    fig.update_layout(scene_aspectmode='data')
    return fig

## Eliminate rotation from 1 spring - 2D

In [ ]:
theta = np.pi/4

In [ ]:
x = np.cos(theta)
y = np.sin(theta)

In [ ]:
diag = np.eye(2)*np.array([x, y])

In [ ]:
diag

In [ ]:
H = np.empty((4, 4))

In [ ]:
H[:2,:2] = diag
H[2:,2:] = diag

In [ ]:
H[:2,2:] = -diag
H[2:,:2] = -diag

In [ ]:
def inertiaAxes(x,y):
    I = np.empty((2,2))
    
    I[0,0] = np.sum(y**2)
    I[1,1] = np.sum(x**2)
    
    I[0,1] = I[1,0] = -np.sum(y*x)
    
    return I

In [ ]:
R = np.array([
    [-.5, -.5],
    [.5, .5]
])

In [ ]:
I = inertiaAxes(R[:,0], R[:,1])

In [ ]:
I_prime, X = np.linalg.eigh(I)

In [ ]:
np.allclose(I, X @ (I_prime*np.eye(2)) @ X.T)

In [ ]:
D = np.empty((3, 4))

In [ ]:
for j in range(2):
    for i in range(2):
        D[2,j*2+i] = np.dot(R[i],X[0])*X[j,1]-np.dot(R[i],X[1])*X[j,0]

In [ ]:
for i in range(len(R)):
    D[:2,i*2:i*2+2] = np.eye(2)

In [ ]:
D.round(3)

In [ ]:
l, v = np.linalg.eigh(H)

In [ ]:
v = v.T

In [ ]:
H.round(3)

In [ ]:
l.round(3), v.round(3)

In [ ]:
for i in range(len(D)):
    norm = np.sum(np.power(D[i],2))
    if norm>1e-3:
        D[i] = D[i]/np.sqrt(norm)

In [ ]:
v

In [ ]:
for i in range(len(v)):
    r = ((D @ v.T).T)[i]
    remainder = v[i] - np.sum(r[:,np.newaxis]*D,0)
    print(remainder.round(3))
    if np.linalg.norm(remainder)<1e-2:
        v[i] = np.zeros(len(v[i]))
    else:
        remainder /= np.linalg.norm(remainder)
        v[i] = remainder
        D = np.append(D, remainder.reshape(1,-1), 0)

In [ ]:
l.round(4)

In [ ]:
D

In [ ]:
scale = .1

for i in range(len(v)):
    plt.xlim((-.5,1.5))
    plt.ylim((-.5,1.5))
    for j in range(int(len(v)/2)):
        x,y = j*np.cos(theta),j*np.sin(theta)
        plt.arrow(x,y, v[i,j*2]*scale, v[i,j*2+1]*scale, width=.01)        
    plt.show()

In [ ]:
(D @ D.T).round(2)

In [ ]:
l_new, v_new = np.linalg.eigh((D.T @ H @ D)[3:,3:])

In [ ]:
l_new

## 2 spring system

In [ ]:
R = np.array([
    [-1., -1],
    [0, 0],
    [1, -1.],
])

In [ ]:
mySum = np.sum(R, 0)
for i in range(len(R)):
    R[i] = R[i] - mySum/len(R)

In [ ]:
springs = [(0,1),(1,2)]

In [ ]:
H = np.zeros((len(R)*2, len(R)*2))
for i, (start, stop) in enumerate(springs):
    proj = np.abs(R[stop]-R[start])
    proj /= np.linalg.norm(proj)
    
    H[start*2:start*2+2, start*2:start*2+2] += -np.eye(2)*proj
    H[stop*2:stop*2+2, stop*2:stop*2+2] += -np.eye(2)*proj
    
    H[start*2:start*2+2, stop*2:stop*2+2] += np.eye(2)*proj
    H[stop*2:stop*2+2, start*2:start*2+2] += np.eye(2)*proj

In [ ]:
def inertiaAxes(x,y):
    I = np.empty((2,2))
    
    I[0,0] = np.sum(y**2)
    I[1,1] = np.sum(x**2)
    
    I[0,1] = I[1,0] = -np.sum(y*x)
    
    return I

In [ ]:
I = inertiaAxes(R[:,0], R[:,1])

In [ ]:
I.round(3)

In [ ]:
I_prime, X = np.linalg.eigh(I)

In [ ]:
X.round(3)

In [ ]:
I_prime.round(3)

In [ ]:
D = np.empty((3, len(R)*2))

In [ ]:
for i in range(len(R)):
    for j in range(2):
        D[2,i*2+j] = np.dot(R[i],X[0])*X[j,1]-np.dot(R[i],X[1])*X[j,0]

In [ ]:
for i in range(len(R)):
    D[:2,i*2:i*2+2] = np.eye(2)

In [ ]:
l, v = np.linalg.eigh(-H)

In [ ]:
v = v.T

In [ ]:
for i in range(len(D)):
    D[i] /= np.linalg.norm(D[i])

In [ ]:
for i in range(len(v)):
    c = np.array([np.dot(D[j],v[i]) for j in range(len(D))])
    proj = np.sum([c[j]*D[j] for j in range(len(D))], 0)
    proj = v[i] - proj
    norm = np.linalg.norm(proj)
    if norm>1e-4:
        D = np.append(D, (proj/norm).reshape(1,-1), 0)
        v[i] = proj/norm
    else:
        v[i] = 0

In [ ]:
scale = .7

for i in range(len(D)):
    plt.xlim((-2,2.))
    plt.ylim((-2,2.))
    for j in range(len(R)):
        x,y = R[j]
        plt.arrow(x,y, D[i,j*2]*scale, D[i,j*2+1]*scale, width=.1)             

    plt.show()

In [ ]:
scale = .7

for i in range(len(D[3:])):
    plt.xlim((-2,2.))
    plt.ylim((-2,2.))
    for j in range(len(R)):
        x,y = R[j]
        plt.arrow(x,y, D[i+3,j*2]*scale, D[i+3,j*2+1]*scale, width=.1)             

    plt.show()

In [ ]:
l_new, v_new = np.linalg.eigh(-(D.T @ H @ D)[3:,3:])

In [ ]:
# 
A = np.array([.5, -.5, 0, 1, -.5, -.5])

In [ ]:
A /= np.linalg.norm(A)

In [ ]:
(D @ A.reshape(-1,1)).round(3)

In [ ]:
l_new.round(3)

In [ ]:
from ipywidgets import interactive

In [ ]:
l.round(3)

In [ ]:
%matplotlib inline
def sumEigen(c0, c1, c2, c3, c4, c5):
    c = np.array([c0, c1, c2, c3, c4, c5]).reshape(-1,1)
    
    newVec = np.sum(D*c, 0)
    plt.figure(2)
    plt.xlim((-2,2.))
    plt.ylim((-2,2.))
    for i in range(len(R)):
        plt.arrow(R[i,0],R[i,1], newVec[i*2]*scale, newVec[i*2+1]*scale, width=.1)
    
    plt.title('')

interactive_plot = interactive(sumEigen, c0=0., c1=0., c2=0., c3=-.5, c4=0., c5=-.866)
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

In [ ]:
l.round(3)

In [ ]:
D[2].round(3)

In [ ]:
(H @ D[2].reshape(-1,1)).round(3)

In [ ]:
oldV = v[2].reshape(-1,1)
newV = H @ oldV

In [ ]:
olfnewV[0,0]

In [ ]:
newV.round(3)

## Find Hinge in 2 triangles - 2D

In [ ]:
R = np.array([
    [-1, -1],
    [-1, 1],
    [0., 0],
    [1, 1.],
    [1,-1],
])

In [ ]:
mySum = np.sum(R, 0)
for i in range(len(R)):
    R[i] = R[i] - mySum/len(R)

In [ ]:
springs = [(0,1), (0,2), (1,2),
          (2,3), (2,4), (3,4)]

In [ ]:
H = np.zeros((len(R)*2, len(R)*2))
for i, (start, stop) in enumerate(springs):
    proj = R[stop]-R[start]
    
    H[start*2:start*2+2, start*2:start*2+2] += np.eye(2)*proj
    H[stop*2:stop*2+2, stop*2:stop*2+2] += np.eye(2)*proj
    
    H[start*2:start*2+2, stop*2:stop*2+2] += -np.eye(2)*proj
    H[stop*2:stop*2+2, start*2:start*2+2] += -np.eye(2)*proj
    

In [ ]:
def inertiaAxes(x,y):
    I = np.empty((2,2))
    
    I[0,0] = np.sum(y**2)
    I[1,1] = np.sum(x**2)
    
    I[0,1] = I[1,0] = -np.sum(y*x)
    
    return I

In [ ]:
I = inertiaAxes(R[:,0], R[:,1])

In [ ]:
I_prime, X = np.linalg.eigh(I)

In [ ]:
np.allclose(I, X @ (I_prime*np.eye(2)) @ X.T)

In [ ]:
D = np.empty((3, len(R)*2))

In [ ]:
for i in range(len(R)):
    for j in range(2):
        D[2,i*2+j] = np.dot(R[i],X[0])*X[j,1]-np.dot(R[i],X[1])*X[j,0]

In [ ]:
for i in range(len(R)):
    D[:2,i*2:i*2+2] = np.eye(2)

In [ ]:
l, v = np.linalg.eigh(H)

In [ ]:
v = v.T

In [ ]:
for i in range(len(D)):
    norm = np.sum(np.power(D[i],2))
    if norm>1e-3:
        D[i] = D[i]/np.sqrt(norm)

In [ ]:
for i in range(len(v)):
    r = ((D @ v.T).T)[i]
    remainder = v[i] - np.sum(r[:,np.newaxis]*D,0)
#    print(remainder.round(3))
    if np.linalg.norm(remainder)<1e-6:
        v[i] = np.zeros(len(v[i]))
    else:
        remainder /= np.linalg.norm(remainder)
        v[i] = remainder
        D = np.append(D, remainder.reshape(1,-1), 0)

In [ ]:
v.round(1)

In [ ]:
l.round(2)

In [ ]:
scale = .7

for i in range(len(v)):
    plt.xlim((-2,2.))
    plt.ylim((-2,2.))
    for j in range(len(R)):
        x,y = R[j]
        plt.arrow(x,y, v[i,j*2]*scale, v[i,j*2+1]*scale, width=.1)             

    plt.show()

In [ ]:
l[6].round(3)

In [ ]:
l.round(2)

In [ ]:
(D @ D.T).round(2)

In [ ]:
l_new, v_new = np.linalg.eigh((D.T @ H @ D)[3:,3:])

In [ ]:
l_new.round(3)

## Eliminate rotation - 3d - 2 springs

In [ ]:
R = np.array([
    [0, 0, 0],
    [1, 0, 0],
    [.5, np.sqrt(3)/2, 0.],
])

In [ ]:
mySum = np.sum(R, 0)
for i in range(len(R)):
    R[i] = R[i] - mySum/len(R)

In [ ]:
springs = [(0,1), (1,2)]

In [ ]:
H = np.zeros((len(R)*3, len(R)*3))
for i, (start, stop) in enumerate(springs):
    proj = R[stop]-R[start]
    proj = np.nan_to_num(proj/np.linalg.norm(proj))
    
    H[start*3:start*3+3, start*3:start*3+3] += np.eye(3)*proj
    H[stop*3:stop*3+3, stop*3:stop*3+3] += np.eye(3)*proj
    
    H[start*3:start*3+3, stop*3:stop*3+3] += -np.eye(3)*proj
    H[stop*3:stop*3+3, start*3:start*3+3] += -np.eye(3)*proj
    

In [ ]:
def inertiaAxes(x,y,z):
    I = np.empty((3,3))
    
    I[0,0] = np.sum(y**2+z**2)
    I[1,1] = np.sum(x**2+z**2)
    I[2,2] = np.sum(x**2+y**2)
    
    I[0,1] = I[1,0] = -np.sum(y*x)
    I[0,2] = I[2,0] = -np.sum(z*x)
    I[1,2] = I[2,1] = -np.sum(y*z)
    
    return I

In [ ]:
l, v = np.linalg.eigh(H)

In [ ]:
v = v.T

In [ ]:
I = inertiaAxes(R[:,0], R[:,1], R[:,2])

In [ ]:
I_prime, X = np.linalg.eigh(I)

In [ ]:
D = np.empty((6,len(R)*3))

In [ ]:
for i in range(len(R)):
    for j in range(3):
        D[3,i*3+j] = np.dot(R[i],X[1])*X[j,2]-np.dot(R[i],X[2])*X[j,1]
        D[4,i*3+j] = np.dot(R[i],X[2])*X[j,0]-np.dot(R[i],X[0])*X[j,2]  
        D[5,i*3+j] = np.dot(R[i],X[0])*X[j,1]-np.dot(R[i],X[1])*X[j,0]

In [ ]:
for i in range(len(R)):
    D[:3,i*3:i*3+3] = np.eye(3)

In [ ]:
for i in range(len(D)):
    D[i] /= np.linalg.norm(D[i])

In [ ]:
for i in range(len(v)):
    remainder = v[i] - np.sum((D @ v[i]).reshape(-1,1) * D, 0)

    if np.linalg.norm(remainder)<.02:
        v[i] = np.zeros(len(v[i]))
    else:
        remainder /= np.linalg.norm(remainder)
        v[i] = remainder
        D = np.append(D, remainder.reshape(1,-1), 0)

In [ ]:
v.round(2)

In [ ]:
nonZeros = np.asarray([x for x in v if not (x==0).all()])

In [ ]:
visualize_normal_modes(R, nonZeros.T)

In [ ]:
(D @ D.T).round(2)

In [ ]:
l_new, v_new = np.linalg.eigh((D.T @ H @ D)[6:,6:])

In [ ]:
l_new.round(4)

## Eliminate rotation from tetrahedron

In [ ]:
R = np.array([
    np.ones(3),
    [1, -1, -1.],
    [-1, 1, -1],
    [-1, -1, 1]
])

In [ ]:
springs = [(0,1),(0,2),(0,3),(1,2),(1,3),(2,3)]

In [ ]:
mySum = np.sum(R, 0)
for i in range(len(R)):
    R[i] = R[i] - mySum/len(R)

In [ ]:
H = np.zeros((len(R)*3, len(R)*3))
for i, (start, stop) in enumerate(springs):
    proj = R[stop]-R[start]
    proj = np.nan_to_num(proj/np.linalg.norm(proj))
    
    H[start*3:start*3+3, start*3:start*3+3] += np.eye(3)*proj
    H[stop*3:stop*3+3, stop*3:stop*3+3] += np.eye(3)*proj
    
    H[start*3:start*3+3, stop*3:stop*3+3] += -np.eye(3)*proj
    H[stop*3:stop*3+3, start*3:start*3+3] += -np.eye(3)*proj
    

In [ ]:
def inertiaAxes(x,y,z):
    I = np.empty((3,3))
    
    I[0,0] = np.sum(y**2+z**2)
    I[1,1] = np.sum(x**2+z**2)
    I[2,2] = np.sum(x**2+y**2)
    
    I[0,1] = I[1,0] = -np.sum(y*x)
    I[0,2] = I[2,0] = -np.sum(z*x)
    I[1,2] = I[2,1] = -np.sum(y*z)
    
    return I

In [ ]:
l, v = np.linalg.eigh(H)

In [ ]:
v = v.T

In [ ]:
I = inertiaAxes(R[:,0], R[:,1], R[:,2])

In [ ]:
I_prime, X = np.linalg.eigh(I)

In [ ]:
D = np.empty((6,len(R)*3))

In [ ]:
for i in range(len(R)):
    for j in range(3):
        D[3,i*3+j] = np.dot(R[i],X[1])*X[j,2]-np.dot(R[i],X[2])*X[j,1]
        D[4,i*3+j] = np.dot(R[i],X[2])*X[j,0]-np.dot(R[i],X[0])*X[j,2]  
        D[5,i*3+j] = np.dot(R[i],X[0])*X[j,1]-np.dot(R[i],X[1])*X[j,0]

In [ ]:
for i in range(len(R)):
    D[:3,i*3:i*3+3] = np.eye(3)

In [ ]:
for i in range(len(D)):
    D[i] /= np.linalg.norm(D[i])

In [ ]:
for i in range(len(v)):
    remainder = v[i] - np.sum((D @ v[i]).reshape(-1,1) * D, 0)

    if np.linalg.norm(remainder)<.02:
        v[i] = np.zeros(len(v[i]))
    else:
        remainder /= np.linalg.norm(remainder)
        v[i] = remainder
        D = np.append(D, remainder.reshape(1,-1), 0)

In [ ]:
nonZeros = np.asarray([not (x==0).all() for x in v])

In [ ]:
l.round(3)

In [ ]:
l[nonZeros].round(3)

In [ ]:
visualize_normal_modes(R, v[nonZeros].T)

In [ ]:
(D @ D.T).round(2)

In [ ]:
l_new, v_new = np.linalg.eigh((D.T @ H @ D)[6:,6:])

In [ ]:
l_new.round(4)